In [1]:
import homer
import glob
import pandas as pd


In [2]:
tw_files='/Users/houghton/Desktop/tw/posts_sample_201506*.gz'    

In [5]:
wel_files = homer.build_weighted_edgelist_db(
    tw_file_globstring='/Users/houghton/Desktop/tw/posts*', 
    intermediate_files_globstring='/Users/houghton/Desktop/tw/hashtags_only/parsing/hashtags_int_*.csv',
    output_files_globstring='/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_*.csv',
    languages=['en'],
    dates=[20150616, 20150617, 20150618, 20150619, 20150620, 20150621, 20150622, 20150623, 20160624],
    threshold=2,
    hashtags_only=True
    )

[########################################] | 100% Completed | 34min 10.9s
[########################################] | 100% Completed |  2min  7.2s


In [3]:
wel_files = glob.glob('/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel*.csv')
wel_files

['/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150616.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150617.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150618.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150619.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150620.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150621.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150622.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20150623.csv',
 '/Users/houghton/Desktop/tw/hashtags_only/hashtag_wel_20160624.csv']

In [4]:
assert len(wel_files) == len([20150616, 20150617, 20150618, 20150619, 20150620, 20150621, 20150622, 20150623, 20160624])

In [5]:
cluster_files = homer.build_cluster_db(
    weighted_edge_list_files=sorted(wel_files)[:-1],
    intermediate_files_directory='/Users/houghton/Desktop/tw/hashtags_only/cos_working',
    output_files_directory='/Users/houghton/Desktop/tw/hashtags_only/cos_output',
    dates=[20150616, 20150617, 20150618, 20150619, 20150620, 20150621, 20150622, 20150623],
    thresholds=[2]
)

In [11]:
cluster_files_df = pd.DataFrame(cluster_files)
cluster_files_df['k'] = cluster_files_df['k'].apply(int)
cluster_files_df.head()

,date,file,k,threshold
0,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,10,2
1,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,11,2
2,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,12,2
3,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,13,2
4,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,14,2


In [2]:
output_files_directory='/Users/houghton/Desktop/tw/hashtags_only/cos_output'
collector = []
for date_dir in glob.glob(output_files_directory+'/*'):
    date = date_dir.split('/')[-1]
    for th_dir in glob.glob(date_dir+'/*'):
        th = th_dir.split('/')[-1]
        for cluster_file in glob.glob(th_dir+'/*.csv'):
            k = int(cluster_file.split('/')[-1].split('_')[0])
            collector.append({'date':date, 'threshold':th, 'k':k, 'file':cluster_file})
            
cluster_files_df = pd.DataFrame(collector)
cluster_files_df.head()

,date,file,k,threshold
0,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,10,2
1,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,11,2
2,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,12,2
3,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,13,2
4,20150616,/Users/houghton/Desktop/tw/hashtags_only/cos_o...,14,2


In [ ]:
import pandas as pd
import dask


def find_k_children(parent_file, child_file):
    parents = pd.read_csv(parent_file, header=None, names=['ID', 'set'], index_col=['ID'])
    parents['set'] = parents['set'].apply(lambda x: set(x.split(' ')))
    child_candidates = pd.read_csv(child_file, header=None, names=['ID', 'set'], index_col=['ID'])
    child_candidates['set'] = child_candidates['set'].apply(lambda x: set(x.split(' ')))

    collector = []
    for parent_id, parent in parents.iterrows():
        children = [child_id for child_id, child in child_candidates.iterrows() if
                    child['set'].issubset(parent['set'])]
        if len(children) > 0:
            collector.append({'ID': parent_id,
                              'children': children})

    #return pd.DataFrame(collector)
    return collector


lazy_find_k_children = dask.delayed(find_k_children)


def build_day(cluster_files_df, output_file, root):
    current_group = cluster_files_df.sort(columns='k').reset_index(drop=True)
    dask_collector = []
    for ia, ib in zip(current_group.index[:-1], current_group.index[1:]):
        dask_collector.append(lazy_find_k_children(cluster_files_df['file'].loc[ia],
                                                   cluster_files_df['file'].loc[ib]))

    for element in dask.bag.from_delayed(dask_collector):
        root
    
    
    template = pd.DataFrame([{'ID': 'nice str', 'children': 'difficult str'}],
                            columns=['ID', 'children'])

    dask.bag.from_delayed(dask_collector).to_file(output_file)


def build_children_db(cluster_files_df, output_globstring, root):
    """
    Identifies children (ie, completely contained clusters)




    Parameters
    ----------
    clusters

    Returns
    -------

    """

    children_filenames_collector = []

    for (date, threshold), group in cluster_files_df.groupby(by=['date', 'threshold']):
        children_filename = output_globstring.replace('*', '_%s_%s' % (date, threshold))
        build_day(group, children_filename, root)
        children_filenames_collector.append(children_filename)

    return children_filenames_collector



In [ ]:
root = homer.tree.Cluster('__root__')


    for ID, row in clusters.iterrows():
        new = Cluster(contents=str(ID),
                      k=row['k'],
                      w=row['threshold'],
                      date=row['Date'])

        root.insert(new)
        if row['k'] == 3:
            root.k_children.append(new)
    
    

In [ ]:

def compute_tree(clusters, relations, transitions, tree_filename):
    # put clusters in a tree
    root = Cluster('__root__')
    for ID, row in clusters.iterrows():
        new = Cluster(contents=str(ID),
                      k=row['k'],
                      w=row['threshold'],
                      date=row['Date'])

        root.insert(new)
        if row['k'] == 3:
            root.k_children.append(new)

    # add clusters as children
    for (_, _, ID), row in relations.iterrows():
        node = root.find('_'+str(ID))
        for child in row['children']:
            node.k_children.append(root.find(str(child)))

    # add leaves (words)
    for node in walk_k_ancestry(root, 'bottom up'):
        if node.contents != '__root__':
            present_in_children = [m.contents for m in node.get_k_members()]
            words = clusters['Set'].loc[int(node.contents[1:])].compute().values[0].split(
                ' ')
            for leaf_word in list(set(words) - set(present_in_children)):
                leaf = root.find(leaf_word)
                if leaf is None:
                    leaf = Cluster(leaf_word, is_leaf=True)
                    root.insert(leaf)
                node.k_children.append(leaf)

    # add connections to subsequent day clusters
    for i, (c1, c2, similarity) in transitions.iterrows():
        n1 = root.find('_'+str(int(c1)))
        n2 = root.find('_'+str(int(c2)))
        n1.tomorrow.append(n2)
        n1.p_tomorrow.append(similarity)

    with gzip.open(tree_filename, 'wb') as f:
        for node in walk_tree(root):
            f.write((node.to_json() + '\n').encode())

    return root

In [3]:
homer.relate.build_children_db(cluster_files_df, '/Users/houghton/Desktop/tw/hashtags_only/hashtag_children*.json')

/Users/houghton/Google Drive/Academic Projects/HOMER/homer/homer/relate.py:27: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  current_group = cluster_files_df.sort(columns='k').reset_index(drop=True)


AttributeError: 'Bag' object has no attribute 'to_file'

In [ ]:
%debug


> /Users/houghton/anaconda/envs/py3k/lib/python3.5/site-packages/pandas/core/indexing.py(1405)error()
   1403                                     "key")
   1404                 raise KeyError("the label [%s] is not in the [%s]" %
-> 1405                                (key, self.obj._get_axis_name(axis)))
   1406 
   1407             try:

